This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'R7AwBpK-3SsWyvXaHtss'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [50]:
# First, import the relevant modules
import requests
import statistics as stats

In [14]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
params = {"start_date": "2019-12-04", "end_date": "2019-12-04", "api_key": API_KEY}
response = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X", params=params)

In [15]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
response.json()

{'dataset': {'collapse': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'data': [['2019-12-04',
    108.6,
    110.4,
    108.5,
    109.6,
    None,
    91446.0,
    10015437.0,
    None,
    None,
    None]],
  'database_code': 'FSE',
  'database_id': 6129,
  'dataset_code': 'AFX_X',
  'description': 'Stock Prices for Carl Zeiss Meditec (2019-11-20) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'end_date': '2019-12-04',
  'frequency': 'daily',
  'id': 10095370,
  'limit': None,
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'newest_available_date': '2019-12-04',
  'oldest_available_date': '2000-06-07',
  'order': None,
  'premium': False,
  'refreshed_at': '2019-12-04T22:18:40.512Z',
  'start_date': '2019-12-04',
  'transform': None,
  'type': 'Time Series'}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [18]:
# 1. Get data for the whole year 2017
params = {"start_date": "2017-01-01", "end_date": "2017-12-31", "api_key": API_KEY}
response = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X", params=params)

In [20]:
response_dict = response.json()
data = response_dict['dataset']['data']

In [27]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.
opening_prices = [day[1] for day in data if day[1]]
print(f"Max: {max(opening_prices)}")
print(f"Min: {min(opening_prices)}")

Max: 53.11
Min: 34.0


In [32]:
# 4 What was the largest change in any one day (based on High and Low price)?
day_diff = [day[2] - day[3] for day in data]
print(f"Biggest Price Change: {max(day_diff)}")

Biggest Price Change: 2.8100000000000023


In [48]:
# 5 What was the largest change between any two days (based on Closing Price)?
closing_prices = [day[4] for day in data if day[4]]
max_diff = 0.0
prev_price = closing_prices[0] # Get the first one to start the ball rolling
for price in closing_prices[1:]:
    max_diff = max(max_diff, round(prev_price-price, 2))
    prev_price = price
print(f"Max Difference: {max_diff}")

Max Difference: 1.72


In [54]:
# What was the average daily trading volume during this year?
trading_volume = [day[6] for day in data if day[6]]
print(f"Average trading volume: {stats.mean(trading_volume)}")

Average trading volume: 89124.33725490196


In [55]:
# What was the average daily trading volume during this year? No stats
trading_volume = [day[6] for day in data if day[6]]
avg = sum(trading_volume)/len(trading_volume)
print(f"Average trading volume: {avg}")

Average trading volume: 89124.33725490196


In [67]:
# (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.
trading_volume = [day[6] for day in data if day[6]]
def median(values):
    ordered = sorted(values)
    mid = len(ordered)//2
    if len(ordered) % 2 == 1:
        return ordered[mid]
    else:
        return (ordered[mid] + ordered[mid-1])/2
print(f"Median Trading volume: {median(trading_volume)}")    

Median Trading volume: 76286.0


In [70]:
# (Optional with stats) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.
trading_volume = [day[6] for day in data if day[6]]
print(f"Median Trading volume: {stats.median(trading_volume)}")

Median Trading volume: 76286.0
